In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import time
import csv
import os
import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import time
import csv
import os
import datetime

# Set up the Selenium WebDriver
driver = webdriver.Chrome()  # or webdriver.Firefox(), etc.

# Load the webpage
url = r"https://fbref.com/en/comps/9/stats/Premier-League-Stats"
driver.get(url)

# Allow time for the page to load and JavaScript to execute
time.sleep(5)  # Adjust the time as necessary based on your internet speed

# Get the page source after JavaScript has executed
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Find the table by examining the content after JS execution
table = soup.find('table', id='stats_standard')

if table is None:
    available_tables = [t['id'] for t in soup.find_all('table') if t.has_attr('id')]
    raise ValueError(f"Table with specified ID not found. Available tables: {available_tables}")

# Specify the columns you want to extract (based on data-stat attributes)
columns_to_extract = ['player', 'age', 'team', 'position', 'nationality']

# Debug: Print the column headers to ensure correct extraction
headers = table.find('thead').find_all('th')
for header in headers:
    print(f"Header: {header.text.strip()}, data-stat: {header.get('data-stat')}")

# Extract player data
players = []
for row in table.find('tbody').find_all('tr'):
    # Skip rows with the 'thead' class (usually section headers within the table)
    if 'thead' in row.get('class', []):
        continue
    
    player_data = {}
    cells = row.find_all('td')
    if len(cells) == 0:  # Skip empty rows
        continue
    
    # Debug: Print cell data before processing
    for cell in cells:
        data_stat = cell.get('data-stat')
        print(f"Cell data-stat: {data_stat}, value: {cell.text.strip()}")

    for cell in cells:
        data_stat = cell.get('data-stat')
        if data_stat in columns_to_extract:
            player_data[data_stat] = cell.text.strip()

    # Debug: Print player data to ensure correct extraction
    if all(col in player_data for col in columns_to_extract):
        print(f"Player data: {player_data}")
        players.append(player_data)

# Save the data to a JSON file with UTF-8 encoding
with open('players.json', 'w', encoding='utf-8') as f:
    json.dump(players, f, indent=4)

# Save the data to a CSV file with UTF-8 encoding
try:
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_file_path = f'players_{timestamp}.csv'
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=columns_to_extract)
        writer.writeheader()
        writer.writerows(players)
except PermissionError as e:
    print(f"PermissionError: {e}. Please check if the file is open or if you have write permissions.")
except UnicodeEncodeError as e:
    print(f"UnicodeEncodeError: {e}. There was an error encoding the file. Check for unsupported characters.")
    # Optionally: Print problematic player data
    for player in players:
        for key, value in player.items():
            if not value.isascii():  # If the value contains non-ASCII characters
                print(f"Problematic value for {key}: {value}")

# Print the extracted data
print(json.dumps(players, indent=4))

# Close the browser
driver.quit()




Header: , data-stat: 
Header: Playing Time, data-stat: header_playing
Header: Performance, data-stat: header_performance
Header: Expected, data-stat: header_expected
Header: Progression, data-stat: header_progression
Header: Per 90 Minutes, data-stat: header_per90
Header: , data-stat: None
Header: Rk, data-stat: ranker
Header: Player, data-stat: player
Header: Nation, data-stat: nationality
Header: Pos, data-stat: position
Header: Squad, data-stat: team
Header: Age, data-stat: age
Header: Born, data-stat: birth_year
Header: MP, data-stat: games
Header: Starts, data-stat: games_starts
Header: Min, data-stat: minutes
Header: 90s, data-stat: minutes_90s
Header: Gls, data-stat: goals
Header: Ast, data-stat: assists
Header: G+A, data-stat: goals_assists
Header: G-PK, data-stat: goals_pens
Header: PK, data-stat: pens_made
Header: PKatt, data-stat: pens_att
Header: CrdY, data-stat: cards_yellow
Header: CrdR, data-stat: cards_red
Header: xG, data-stat: xg
Header: npxG, data-stat: npxg
Header: 